In [ ]:
# MINERIA DE DATOS
# Integrantes del equipo
# Hernandez Perez Jorge Luis - 201852774
# Huerta Hernandez Yusdivia -  201843995
# Juarez Martines Guillermo - 201873517
# Tzompantzi Nahuatlato Juan Carlos - 201854502

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input/kmeans/analisis.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# ***Agrupar usuarios en Twitter de acuerdo a su personalidad con k-means***

In [ ]:
#Comenzaremos importando las librerías que nos asistirán para ejecutar el algoritmo y graficar.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
 
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

In [ ]:
#Importamos el archivo csv - para simplificar,
#el archivo se encuentra en el mismo directorio que el notebook- 
#y vemos los primeros 5 registros del archivo tabulados.
dataframe = pd.read_csv(r"/kaggle/input/kmeans/analisis.csv")
dataframe.head(5)

In [ ]:
#También podemos ver una tabla de información estadística que nos provee Pandas dataframe: Python
dataframe.describe()

In [ ]:
#El archivo contiene diferenciadas 9 categorías -actividades laborales.
#Para conocer cuantos registros tenemos de cada ponemos las siguientes lineas
print(dataframe.groupby('categoria').size())

In [ ]:
#Veremos graficamente nuestros datos para tener una idea de la dispersión de los mismos
dataframe.drop(['categoria'],1).hist()
plt.show()

In [ ]:
#seleccionamos 3 dimensiones: op, ex y ag 
#las cruzamos para ver si nos dan alguna pista de su agrupación y la relación con sus categorías.
sb.pairplot(dataframe.dropna(), hue='categoria',size=4,vars=["op","ex","ag"],kind='scatter')

In [ ]:
#Definimos la entrada
#Concretamos la estructura de datos que utilizaremos para alimentar el algoritmo.
#Sólo cargamos las columnas op, ex y ag en nuestra variable X.
X = np.array(dataframe[["op","ex","ag"]])
y = np.array(dataframe['categoria'])
X.shape

In [ ]:
#Tenemos una gráfica en 3D con 9 colores representando las categorías.
fig = plt.figure()
ax = Axes3D(fig)
colores=['blue','red','green','blue','cyan','yellow','orange','black','pink','brown','purple']
asignar=[]
for row in y:
    asignar.append(colores[row])
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=asignar,s=60)

In [ ]:
#Obtener el valor K
#Hallamos el valor de K haciendo una gráfica
#intentando hallar el “punto de codo”
Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
kmeans
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
score
plt.plot(Nc,score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
#Ejecutamos K-Means
#Ejecutamos el algoritmo para 5 clusters y obtenemos las etiquetas y los centroids.
kmeans = KMeans(n_clusters=5).fit(X)
centroids = kmeans.cluster_centers_
print(centroids)

In [ ]:
#Ahora sepuede ver esto en una gráfica 3D con colores para los grupos
#y veremos si se diferencian: (las estrellas marcan el centro de cada cluster)

# Predicting the clusters
labels = kmeans.predict(X)
# Getting the cluster centers
C = kmeans.cluster_centers_
colores=['red','green','blue','cyan','yellow']
asignar=[]
for row in labels:
    asignar.append(colores[row])
 
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=asignar,s=60)
ax.scatter(C[:, 0], C[:, 1], C[:, 2], marker='*', c=colores, s=1000)
#odemos ver que el Algoritmo de K-Means con K=5 ha agrupado a los 140 usuarios
#Twitter por su personalidad, teniendo en cuenta las 3 dimensiones que utilizamos:
#Openess, Extraversion y Agreeablenes. 
#Pareciera que no hay necesariamente una relación en los grupos con sus actividades de Celebrity.

In [ ]:
#Se tienen/generan 3 gráficas en 2 dimensiones
#con las proyecciones a partir de nuestra gráfica 3D 
#para que nos ayude a visualizar los grupos y su clasificación

#GRAFICA 1

# Getting the values and plotting it
f1 = dataframe['op'].values
f2 = dataframe['ex'].values
 
plt.scatter(f1, f2, c=asignar, s=70)
plt.scatter(C[:, 0], C[:, 1], marker='*', c=colores, s=1000)
plt.show()

In [ ]:
#GRAFICA 2
# Getting the values and plotting it
f1 = dataframe['op'].values
f2 = dataframe['ag'].values
 
plt.scatter(f1, f2, c=asignar, s=70)
plt.scatter(C[:, 0], C[:, 2], marker='*', c=colores, s=1000)
plt.show()

In [ ]:
#GRAFICA 3
f1 = dataframe['ex'].values
f2 = dataframe['ag'].values
 
plt.scatter(f1, f2, c=asignar, s=70)
plt.scatter(C[:, 1], C[:, 2], marker='*', c=colores, s=1000)
plt.show()

#Se puede observar que están bastante bien diferenciados los grupos.

In [ ]:
#Ahora bien, podemos ver cada uno de los clusters cuantos usuarios tiene:
copy =  pd.DataFrame()
copy['usuario']=dataframe['usuario'].values
copy['categoria']=dataframe['categoria'].values
copy['label'] = labels;
cantidadGrupo =  pd.DataFrame()
cantidadGrupo['color']=colores
cantidadGrupo['cantidad']=copy.groupby('label').size()
cantidadGrupo

In [ ]:
#De igual manera podemos ver la diversidad en rubros laborales de cada uno.
#Por ejemplo en el grupo 0 (rojo), vemos que hay de todas las actividades
#laborales aunque predominan de actividad 1 y 2 correspondiente a 
#Actores y Cantantes con 11 y 15 famosos
group_referrer_index = copy['label'] ==0
group_referrals = copy[group_referrer_index]
 
diversidadGrupo =  pd.DataFrame()
diversidadGrupo['categoria']=[0,1,2,3,4,5,6,7,8,9]
diversidadGrupo['cantidad']=group_referrals.groupby('categoria').size()
diversidadGrupo
#De categoría 3 “modelos” hay 6 sobre un total de 9.

In [ ]:
# Se va a buscarlos usuarios que están 
#más cerca a los centroids de cada grupo que podríamos decir que
#tienen los rasgos de personalidad característicos que representan a cada cluster:

#vemos el representante del grupo, el usuario cercano a su centroid
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
closest
#El resultado de la consulta muestra la posicion en el array de usuarios

In [ ]:
#En este caso podemos ver que en los centros tenemos una modelo,
#un político, presentadora de Tv, locutor de Radio y un deportista.
users=dataframe['usuario'].values
for row in closest:
    print(users[row])


In [ ]:
#Clasificar nuevas muestras
#Finalmente podemos agrupar y etiquetar nuevos usuarios
#twitter con sus características y clasificarlos.
X_new = np.array([[45.92,57.74,15.66]]) #davidguetta
 
new_labels = kmeans.predict(X_new)
print(new_labels)
#Vemos el ejemplo con el usuario de David Guetta y nos devuelve que pertenece al grupo 2 

In [ ]:
#Metrica taxista
#La Métrica del Taxista está estrechamente relacionada con la distancia recorrida por un taxista o un caminante en una ciudad ordenada y bien planificada, aunquedifícil de encontrar, donde las trayectorias sólo tienen dos sentidos de Norte a Sur (NS) y de Oeste a Este (OE)

#Metrica Completeness
#Indica si el SI contiene toda la información de interés, recubre aspectos extensionales (cantidad de entidades/estados de la realidad representados) e intencionales (cantidad de datos sobre cada entidad/estado) del SI. Para la completitud se definen los siguientes factores: cobertura (coverage) y densidad (density). La cobertura es la porción de los datos de la realidad contenidos en el SI, lo cual implica una comparación del SI con el mundo real que suele ser costosa. Como alternativa se plantea estimar el tamaño que tendría un referencial. La densidad mide cuánta información se tiene sobre las entidades del SI.

#Metrica Harmonia
#Equilibrio, proporción y correspondencia adecuada entre las diferentes cosas de un conjunto.

#Metrica homogeneidad
#Los mecanismos de homogeneidad permiten agregar seguridad y representatividad a los datos que se analizan, permitiendo únicamente fluctuaciones que representen cambios por efectos naturales